## Health Care for All Case Study using Pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
## Install xlrd package to load Excel files
# conda install openpyxl
## conda install xlrd

In [ ]:
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train

In [ ]:
from scipy.stats import iqr
def remove_outliers(df):
    for c in df.columns:
            pct_75 = np.percentile(df[c], 75)
            pct_25 = np.percentile(df[c], 25)
            upper_bound = pct_75 + 1.5*iqr(df[c])
            lower_bound = pct_25 - 1.5*iqr(df[c])
            condition = (df[c] < upper_bound) & (df[c] > lower_bound)
            df[c] = df[c][condition]  # Filter out the outliers
    return df

<b> We begin by loading the precleaned data set

In [ ]:
hk_df= pd.read_csv("hk_df_cleaned.csv")

In [ ]:
hk_df.columns # inspect the column names

<b> checking correlations between numerical variables

In [ ]:
sns.pairplot(hk_df.select_dtypes(np.number)) # pairplot generates a grid of scatter plots for all numerical variables except on the diagonal where it displays distributions

In [ ]:
plt.scatter(x=hk_df['med_fam_income'], y=hk_df['avg_household_income'],c='g') # note the correlation between income sources


<b> Plotting the correlation heatmap

In [ ]:
corr = hk_df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True # trick to filter out the upper-right triangle, which is redundant due to symmetry
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(16, 14))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

<b> Removing highly correlated columns


In [ ]:
CORR_THRESH = 0.80
corr_matrix=hk_df.corr().abs()
upper_triangle=corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))
corrd_cols = [column for column in upper_triangle.columns if any(upper_triangle[column] > CORR_THRESH)]
hk_df.drop(corrd_cols,axis=1,inplace=True)
hk_df.columns

In [ ]:
hk_df.hist(figsize=(11,12))

<b> Distribution of "median home value" is skewed towards lower incomes

In [ ]:
sns.displot((hk_df['median_home_val']), bins=20)

## X,y, one-hot, and test/train

In [ ]:
X = hk_df.drop('target_d', axis=1)
y = hk_df.target_d

In [ ]:
numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

In [ ]:
numericalX

In [ ]:
med=pd.DataFrame(hk_df.median_home_val)

In [ ]:
y.isna().any()

In [ ]:
med.hist()

In [ ]:
y.dropna(inplace=True)

In [ ]:
y

In [ ]:
y = pd.DataFrame(y)

In [ ]:
remove_outliers(pd.DataFrame(y))

In [ ]:
# one-hot encode the categorical features
X = pd.concat([pd.get_dummies(X[categoricalX.columns],drop_first=True),
               remove_outliers(X[numericalX.columns])],
              axis=1)

In [ ]:
X.isna().any()

In [ ]:
hk_df.columns

In [ ]:
na_idcs = X[X.isna().any(axis=1)].index
X = pd.DataFrame(X).drop(na_idcs)
y = pd.DataFrame(y).drop(na_idcs)

In [ ]:
na_idcs_y = y[y.isna().any(axis=1)].index
X = pd.DataFrame(X).drop(na_idcs_y)
y = pd.DataFrame(y).drop(na_idcs_y)

In [ ]:
y.isna().any()

In [ ]:
# test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TT_RATIO, random_state=RAND_STATE)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.head(3)

## Continuous transformations

<b> Power Transformer </b>: we use a power transformer to make the distribution of this variable more normal-like.  It improves modeling errors in linear regression.

In [ ]:
pt = PowerTransformer()
med_home_val_transformed=pt.fit_transform(hk_df['median_home_val'].to_numpy().reshape(-1,1))
sns.displot(med_home_val_transformed)

In [ ]:
print("the parameters used to transform median_home_val are")
pt.get_params(),pt.lambdas_ # parameter used in the power transformation

In [ ]:
X_train

In [ ]:
ct = ColumnTransformer([("pt", pt, list(numericalX.columns))],
                        remainder='drop', verbose=True).fit(X_train)
X_train_ct = pd.DataFrame(ct.transform(X_train))
X_test_ct = pd.DataFrame(ct.transform(X_test))

In [ ]:
X_train_ct = pd.DataFrame(X_train_ct)
X_test_ct

## Predictive Modeling 

### OLS using StatsModels

In [ ]:
y_train

In [ ]:
X_train_const_ct = sm.add_constant(X_train_ct.to_numpy()) # adding a constant

model = sm.OLS(y_train, X_train_const_ct).fit()
predictions_train = model.predict(X_train_const_ct)

X_test_const_ct = sm.add_constant(X_test_ct) # adding a constant
predictions_test = model.predict(X_test_const_ct)
print_model = model.summary()
print(print_model)

### OLS using Scikit Learn

Model fitting

In [ ]:
model=LinearRegression()    # model
model.fit(X_train_ct, y_train)   # model train

<b> model parameters

In [ ]:
model.coef_

In [ ]:
model.intercept_

Making prediction

In [ ]:
y_pred = pd.DataFrame(model.predict(X_test_ct),columns = ['target_d'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(X_train_ct),columns = ['target_d'])

## Evaluating Model Performance

In [ ]:
y_pred

In [ ]:
result=pd.DataFrame({"y_test": list(y_test['target_d']),"y_pred": list(y_pred['target_d'])})

In [ ]:
result

In [ ]:
# Make a scatterplot of y_pred vs y
# Question: What kind of plot will you get if all the all the predictions are perfect?
# Answer: A straight line!

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1].hist(y_test - y_pred)
ax[1].set_xlabel("Test y-y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred,y_pred.to_numpy()-y_test.to_numpy(),"o")
ax[2].set_xlabel("predited")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred,np.zeros(len(y_pred)),linestyle='dashed')

<b>more fancy using seaborn

In [ ]:
yp_ = y_pred.to_numpy()
yt_ = y_test.to_numpy()
sns.regplot(x=yp_,y=yt_,scatter_kws={"color": "red"}, line_kws={"color": "black"})

### Error metrics

In [ ]:
print(mse(y_test,y_pred))
print(mae(y_test,y_pred))
##prediction on the train set
print(mse(y_train,y_pred_train))

In [ ]:
R2=r2_score(y_test,y_pred)
R2

In [ ]:
R2_test=model.score(X_test_ct,y_test)
R2_train=model.score(X_train_ct,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
Adj_R2

## Feature Importances

In [ ]:
feature_importances = pd.DataFrame(data={
    'Variable': X_train_ct.columns,
    'Importance': abs(model.coef_.reshape(len(X_train_ct.columns),))
})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
feature_importances

In [ ]:
plt.bar(x=features_importances['Variable'].iloc[:10], height=features_importances['Importance'].iloc[:10], color='#087E8B')
plt.title('Feature importance rankings', size=12)
plt.xticks(rotation='horizontal')
plt.show()